## Reed-Muller Code:

In [1]:
class ReedMullerCode:
    
    def __init__(self, r, m, q):
        
        if not (q == 2):
            raise ValueError('Works only in binary case')
            
        self.r = r
        self.m = m
        self.q = q
        
        self.n = 2**self.m
        self.k = sum(binomial(self.m,i) for i in range(self.r + 1))
        self.d = 2**(self.m - self.r)
        
        # Initializing field
        self.F = FiniteField(q)
        
        self.G = matrix(self.F, self.k, self.n, lambda i,j : 0)
        
        print('n, k, d:')
        print(self.n,self.k,self.d)
        self.G = self.get_G(self.r, self.m)
        print(self.G)
        
    def Encoding(self, message, zeropad = True):
        
        rem = len(message) % self.k
        
        if rem != 0:
            if zeropad:
                message.extend([self.F(0)]*(self.k-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(message), self.k):
            c.extend(self.EncodeChunk(message[i:i+self.k]))
        
        return c
            
    def EncodeChunk(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.F, chunk) * self.G

        return c
        
    
    def Decoding(self, received):
        
        # Check input size
        if len(received) % self.n != 0:
            raise ValueError('Invalid input size')
            
        c = []
        
        for i in range(0,len(received),self.n):
            c.extend(self.DecodeChunk(received[i:i+self.n], self.r, self.m))
            
        return c
        
            
    def get_G(self, r, m):
        n = 2**m
        k = sum(binomial(m,i) for i in range(r + 1))
        d = 2**(m - r)

        if (n == k):
            # trivial code
            return matrix(self.F, k, n, lambda i,j : 1) #???

        elif (n == d):
            # repetition code
            return matrix(self.F, k, n, lambda i,j : 1)

        elif (d == 2 and n == k + 1):
            # parity check code
            PCC = codes.ParityCheckCode(GF(self.q), k)
            return PCC.generator_matrix()

        else:
            return block_matrix([ [self.get_G(r,m-1), self.get_G(r,m-1)], [0, self.get_G(r-1,m-1)] ])
            
            
    def DecodeChunk(self, word, r, m):
        
        word = vector(self.F, word)
        
        #if len(word) != self.n:
        #    raise ValueError('Invalid chunk size')
        
        if (r == 0):
            return self.repetition_decode(word)
        if (m == r + 1):
            return 'parity check'
        l = len(word)
        u1 = word[:l//2]
        u2_v = word[l//2:]
        v_error = u1 + u2_v

        v = self.DecodeChunk(v_error, r - 1, m - 1)
        u2 = u2_v + v
        
        u1_decoded = self.DecodeChunk(u1, r, m - 1)
        u2_decoded = self.DecodeChunk(u2, r, m - 1)
        if (u1_decoded == 'parity check' and u2_decoded == 'parity check'):
            u = self.parity_check_compare(u1, u2)
        
        #e1 = hw(u1 - u1_decoded) + hw(u2 - u1_decoded)
        #e2 = hw(u1 - u2_decoded) + hw(u2 - u2_decoded)
        
        #if (e1 < e2):
        #    u = u1_decoded
        #else:
        #    u = u2_decoded
        
        #uv = u + v
        return vector(self.F, u.list() + (u + v).list())
    
    def repetition_decode(self, word):
        
        if word.hamming_weight() >= len(word.list())//2:
            return vector(self.F, [1 for i in range(len(word.list()))])
        elif word.hamming_weight() < len(word.list())//2:
            return vector(self.F, [0 for i in range(len(word.list()))])
        else:
            return "Failure"
        
    def parity_check_compare(self, word1, word2):
        decoded1 = word1[:-1]
        decoded2 = word2[:-1]
        
        if (((decoded1.hamming_weight() % 2) == 0 and word1[-1] == 0) or ((decoded1.hamming_weight() % 2) == 1 and word1[-1] == 1)):
            return word1
        elif (((decoded2.hamming_weight() % 2) == 0 and word2[-1] == 0) or ((decoded1.hamming_weight() % 2) == 1 and word2[-1] == 1)):
            return word2
        else:
            return "decoding failure"
        

In [2]:
C = ReedMullerCode(2,4,2)

n, k, d:
16 11 4
[1 0 0 0 0 0 0 1|1 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 1|0 1 0 0 0 0 0 1]
[0 0 1 0 0 0 0 1|0 0 1 0 0 0 0 1]
[0 0 0 1 0 0 0 1|0 0 0 1 0 0 0 1]
[0 0 0 0 1 0 0 1|0 0 0 0 1 0 0 1]
[0 0 0 0 0 1 0 1|0 0 0 0 0 1 0 1]
[0 0 0 0 0 0 1 1|0 0 0 0 0 0 1 1]
[---------------+---------------]
[0 0 0 0 0 0 0 0|1 0 0 1 1 0 0 1]
[0 0 0 0 0 0 0 0|0 1 0 1 0 1 0 1]
[0 0 0 0 0 0 0 0|0 0 1 1 0 0 1 1]
[0 0 0 0 0 0 0 0|0 0 0 0 1 1 1 1]


In [3]:
message = [1,0,1,0,1,0,1,0,1,0,1]
c = C.Encoding(message)
print("codeword\n", c)
M = C.Decoding(c)
print("decoded word\n", M)

codeword
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
decoded word
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
